In [ ]:
%load_ext autoreload
%autoreload 2

import sys

from waterchange.config import read_config
from waterchange.datasets import setup_directory
from waterchange.utils import get_logger

from waterchange.models import (
    predict_average_water_index_all_pixels_from_first_coefs_all_bands_regression,
    create_inference_map,
    predict_average_water_change_at_break_point_from_trained_model,
    predict_average_water_change_at_break_point,
    acc_stat,
    cal_p_value
)

# Add the custom path to the system path
sys.path.append('/home/hpham/code')

get_logger("error")

# Load the config.yaml to get all the directories
config = read_config()
input_data = setup_directory(config)

# Predict water change at break points using trained regression model.

In [ ]:
# Train the regression model - single model
infer_region_id = 1
train_region_id = [x for x in input_data.rois if x != infer_region_id]
logger = get_logger("Debug")
logger.info(f"Train: {train_region_id}, test: {infer_region_id}")

df_infer, model = predict_average_water_index_all_pixels_from_first_coefs_all_bands_regression(
    model_name="gradient_boosting",
    input_data=input_data,
    sub_dir_in_train="water_coefs_in_period",
    sub_dir_in_infer_first_all="first_coefs_all_pixels_to_df",
    sub_dir_in_infer_in_period="water_coefs_in_period",
    train_region_id=train_region_id,
    infer_region_id=infer_region_id)


In [ ]:
get_logger("INFO")

res_infer = []

train_region_id = [x for x in input_data.rois if x != infer_region_id]
_, acc_infer, dist  = predict_average_water_change_at_break_point_from_trained_model(
input_data,
model,
sub_dir_in_train="water_coefs_break_point",
infer_region_id=infer_region_id,
threshold=0.2)

res_infer.append(acc_infer)

In [ ]:
# Cross validate
logger = get_logger("debug")

threshold = 0.1

res_infer = []
distribution = []

for infer_region_id in input_data.rois:
    train_region_id = [x for x in input_data.rois if x != infer_region_id]
    logger.info(f"Train: {train_region_id}, test: {infer_region_id}")

    df_infer, model = predict_average_water_index_all_pixels_from_first_coefs_all_bands_regression(
        model_name="gradient_boosting",
        input_data=input_data,
        sub_dir_in_train="water_coefs_in_period",
        sub_dir_in_infer_first_all="first_coefs_all_pixels_to_df",
        sub_dir_in_infer_in_period="water_coefs_in_period",
        train_region_id=train_region_id,
        infer_region_id=infer_region_id)

    _, acc_infer, dist  = predict_average_water_change_at_break_point_from_trained_model(
    input_data,
    model,
    sub_dir_in_train="water_coefs_break_point",
    infer_region_id=infer_region_id,
    threshold=threshold)

    distribution.append(dist)
    res_infer.append(acc_infer)


In [ ]:
res_infer

In [ ]:
acc_stat(res_infer)

In [ ]:
distribution

In [ ]:
"""
# Results
threshold = 0.1
Overall	    0	    1	    2            %           unis
0.6	    0.445	0.673	0.725   (42,18,40)	(746,318,732)
0.437	0.418	0.536	0.425   (80,16,4)   (3246,658,167)
0.599	0.57	0.603	0.68    (62,17,21)  (430,121,147)
0.677	0.624	0.724	0.87    (60,31,9)   (367,192,54)
0.64	0.573	0.76	0.691   (58,24,18)  (178,75,55)
0.603	0.594	0.4	    0.68    (76,4,20)   (96,5,25)
0.822	0.826	0.5	    0.818   (95,1,4)    (512,6,22)

Regression              Classification
Overall: 0.63 ± 0.11    0.69 ± 0.16
Class 0: 0.58 ± 0.12    0.86 ± 0.15
Class 1: 0.60 ± 0.12    0.35 ± 0.26
Class 2: 0.70 ± 0.13    0.32 ± 0.19
------------------------------
threshold = 0.2
Overall	    0	    1	    2
0.599	0.607	0.557	0.599   (59,12,29)  (1067,210,519)
0.702	0.744	0.406	0.311   (88,10,2)   (3593,404,74)
0.705	0.734	0.678	0.53    (80,8,12)   (556,59,83)
0.79	0.825	0.667	0.789   (73,21,6)   (446,129,38)
0.659	0.673	0.639	0.6     (70,20,10)  (217,61,30)
0.69	0.759	0.25	0.286   (86,3,11)   (108,4,14)
0.911	0.925	1	    0.526   (96,0,4)    (520,1,19)

Regression              Classification
Overall: 0.72 ± 0.09    0.74 ± 0.15
Class 0: 0.75 ± 0.09    0.83 ± 0.15
Class 1: 0.60 ± 0.22    0.39 ± 0.28
Class 2: 0.52 ± 0.16    0.32 ± 0.21
-----------------------------
threshold = 0.3
Overall	    0	    1	    2
0.655	0.71	0.391	0.493   (77,6,16) (1390,110,296)
0.888	0.932	0.239	0.229   (94,5,1) (3814,222,35)
0.787	0.827	0.656	0.385   (88,5,7) (614,32,52)
0.86	0.904	0.646	0.833   (80,16,4) (490,99,24)
0.714	0.756	0.569	0.286   (81,17,2) (250,51,7)
0.833	0.887	0	    0.3     (91,1,8) (115,1,10)
0.944	0.956	1	    0.417   (98,0,2) (527,1,12)

Regression              Classification
Overall: 0.81 ± 0.09    0.77 ± 0.14
Class 0: 0.85 ± 0.09    0.82 ± 0.14
Class 1: 0.50 ± 0.30    0.43 ± 0.30
Class 2: 0.42 ± 0.19    0.31 ± 0.22
"""


# Predict water change using concatenate 112 coefficients

In [ ]:


get_logger("INFO")
threshold = 0.3
res_train = []
res_infer = []
distribution = []
for index in input_data.rois:
    infer_region_id = index
    train_region_id = [x for x in input_data.rois if x != infer_region_id]
    _, acc_train, acc_infer, dist  = predict_average_water_change_at_break_point(
        input_data,
        sub_dir_in_train="water_coefs_break_point",
        train_region_id=train_region_id,
        infer_region_id=infer_region_id,
        threshold=threshold)

    res_train.append(acc_train)
    res_infer.append(acc_infer)
    distribution.append(dist)


In [ ]:
acc_stat(res_infer)

In [ ]:
distribution

In [ ]:
"""
With scaler           Without scaler
Overall: 0.83 ± 0.01  Overall: 0.83 ± 0.01
Class 0: 0.91 ± 0.03  Class 0: 0.91 ± 0.03
Class 1: 0.65 ± 0.04  Class 1: 0.65 ± 0.04
Class 2: 0.67 ± 0.11  Class 2: 0.67 ± 0.11
Overall: 0.69 ± 0.12  Overall: 0.69 ± 0.16
Class 0: 0.96 ± 0.02  Class 0: 0.86 ± 0.15
Class 1: 0.20 ± 0.13  Class 1: 0.35 ± 0.26
Class 2: 0.11 ± 0.12  Class 2: 0.32 ± 0.19
"""